In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import json

In [2]:
!pip install transformers==4.27.4
!pip install wandb
!pip install datasets
!pip install bert_score
!pip install evaluate
!pip install accelerate
!pip install gradio --upgrade
!pip install --upgrade torch
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=872573bb890280218eb420b6dca20649e6c9759f14b013889892b15cf3e5d6bc
  Stored in directory: /root/.cache/pip/wheels

In [3]:
#text preprocessing
def parse_spoiler_for_correct_spoiler_positions(df):
    spoiler = []        
    for sp in df['spoilerPositions']:
        if len(sp) <=1:
          df['spoilerPositions'] = [[[4, 37],[4, 222]]]  # this data is manually checked and assigned because it was throwing error
        else:
          start,end = sp[0],sp[1]
          spoiler.append(df['targetParagraphs'][start[0]][start[1]:end[1]])
        
    return spoiler

In [4]:
#text preprocessing
def find_token_positions(x):    
    token_position = []
    for position in x['spoilerPositions']:
        if len(position) <2:
          print("format of data is not right for uuid", x['uuid'])
        else:
          index = 0
          start,end = position
          
          for idx,element in enumerate([x['targetTitle']] + x['targetParagraphs']):
              if idx==start[0]+1:
                  # because character is starting at given index +1
                  end_index = index + end[1]
                  start_index = index+start[1]
                  token_position.append([start_index,end_index])
                  break
              if idx==0:
                  index+=len(element)+3
              else:
                  index+=len(element)+1
        
    return token_position

In [5]:
#formatting dataframe columns accroding to requirements
def formatted_df(df):
  
  df['spoilerParsed'] = df.apply(parse_spoiler_for_correct_spoiler_positions,1)
  df['combinedParagraphs'] = df['targetParagraphs'].apply(lambda x:" ".join(x),1)
  df.combinedParagraphs = df.targetTitle + " - " + df.combinedParagraphs
  df['tokenPosition'] = df.apply(find_token_positions,1) 
  df['tags'] = df.tags.apply(lambda x:x[0],1)
  df['postText'] = df.postText.apply(lambda x:x[0],1)
  df['label'] = df['tags'].map({"phrase":0,"passage":1,"multi":2})
  return df
    

In [6]:
data_path = './train.jsonl'
df = pd.read_json(data_path, lines=True)
df_valid = pd.read_json("./validation.jsonl", lines=True)
formatted_df(df)
formatted_df(df_valid)

,uuid,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags,spoilerParsed,combinedParagraphs,tokenPosition,label
0,6dc7ddef-4e8e-4a6b-9296-526377518071,800048986762423_885081814925806,Five Nights at Freddy’s Sequel Delayed for Wei...,Facebook,[Five Nights at Freddy’s creator Scott Cawthon...,Five Nights at Freddy’s Sequel Delayed for Wei...,Five Nights at Freddy's creator Scott Cawthon ...,None,[https://gamerant.com/wp-content/uploads/five-...,https://gamerant.com/five-nights-at-freddys-se...,"{'source': 'anonymized', 'humanSpoiler': 'They...",[some of the plot elements are so disturbing t...,"[[[2, 158], [2, 236]]]",passage,[some of the plot elements are so disturbing t...,Five Nights at Freddy’s Sequel Delayed for Wei...,"[[825, 903]]",1
1,435b24de-56f6-4d4e-9c38-54b8e0630aac,4jp20e,Why Arizona Sheriff Joe Arpaio’s fate could ha...,reddit,[© REUTERS/Laura Segall Maricopa County Sherif...,Why Arizona Sheriff Joe Arpaio’s fate could ha...,"<p>A single word — ""intentionally"" — could tra...",None,[https://archive.is/0eGBT/bbf6d79b366fc7e75b8b...,https://archive.is/0eGBT,"{'source': 'anonymized', 'humanSpoiler': '""Int...","[""intentionally"", could transform a court case...","[[[0, 197], [0, 212]], [[0, 215], [0, 328]]]",multi,"[""intentionally"", could transform a court case...",Why Arizona Sheriff Joe Arpaio’s fate could ha...,"[[265, 280], [283, 396]]",2
2,8091ba93-6376-473a-9117-01d6cf0507e4,857559227384160256,Here’s how much you should be tipping your hai...,Twitter,[Here’s how much you should be tipping your ha...,Here’s how much you should be tipping your hai...,Remembering how much you’re supposed to tip an...,,None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[20%],"[[[3, 58], [3, 61]]]",phrase,[20%],Here’s how much you should be tipping your hai...,"[[729, 732]]",0
3,8b713277-0e5b-4873-a216-b650f21f3b4c,389091583213453312,"""Harry Potter"" alums reunite for new movie",Twitter,[The mythology of punk music's evolution can b...,"Alan Rickman & Rupert Grint On 'CBGB,' Reuniti...",The mythology of punk music's evolution can be...,"Alan Rickman,Hilly Kristal,new rupert grint mo...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1ccNwKJ,"{'source': 'anonymized', 'humanSpoiler': 'Alan...","[Alan Rickman & Rupert Grint, CBGB]","[[[-1, 0], [-1, 27]], [[0, 98], [0, 102]]]",multi,"[""CBGB"" is out in limited re, CBGB]","Alan Rickman & Rupert Grint On 'CBGB,' Reuniti...","[[0, 27], [195, 199]]",2
4,a2f91b65-c36c-481f-92b1-2fc77d6411fc,744740593046274048,A man swallowed a microSD card and you won't b...,Twitter,[PetaPixel is one of my favorite blogs. The wr...,Man swallowed a microSD card and you won't bel...,PetaPixel is one of my favorite blogs. The wri...,None,[https://cdn0.vox-cdn.com/uploads/chorus_image...,http://www.theverge.com/circuitbreaker/2016/6/...,"{'source': 'anonymized', 'humanSpoiler': 'This...",[a man who swallowed a 64GB microSD card and t...,"[[[1, 34], [1, 108]]]",passage,[a man who swallowed a 64GB microSD card and t...,Man swallowed a microSD card and you won't bel...,"[[317, 391]]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1189d343-42eb-47e7-8395-ff978a683875,428006164904034305,This is what happens when you leave a hotel cl...,Twitter,[Instead of encountering a mound of dirty towe...,This Is What Happens When You Leave A Hotel Cl...,Instead of encountering a mound of dirty towel...,"givebackfilms,give back films,video,random act...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1ebARdm,"{'source': 'anonymized', 'humanSpoiler': 'She ...",[The video below shows the stunned cleaner ini...,"[[[3, 0], [3, 150]]]",passage,[The video below shows the stunned cleaner ini...,This Is What Happens When You Leave A Hotel Cl...,"[[542, 692]]",1
796,7912282b-137b-4098-875d-8ad9f19354a8,806153730206892032,This Texas GOP elector announces

In [8]:
from transformers import AutoTokenizer
model_name = 'deepset/roberta-base-squad2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
import evaluate
bleu = evaluate.load("bleu",n=4)
squad_metric = evaluate.load("squad")
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [10]:
def generate_new_qa_form_df(df):
    df_new = df[['uuid','targetTitle','postText',"combinedParagraphs","tokenPosition","spoiler"]]
    df_new["answers"] = df_new.apply(lambda x: {'text':x['spoiler'], "answer_start":[x['tokenPosition'][0][0]]},1)
    df_new = df_new.drop(columns=["tokenPosition","spoiler"])
    df_new.columns = ["id","title","question","context","answers"]
    
    return df_new

In [11]:
spoiler_type = "phrase"
train_df = df[df.tags==spoiler_type]
val_df = df_valid[df_valid.tags==spoiler_type]

len(train_df),len(val_df)

(1367, 335)

In [12]:
max_length = 512
stride = 128

#huggingface documentation
def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [13]:
train_df.head(2)

,uuid,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags,spoilerParsed,combinedParagraphs,tokenPosition,label
1,b1a1f63d-8853-4a11-89e8-6b2952a393ec,411701128456593408,NASA sets date for full recovery of ozone hole,Twitter,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1cH672Z,"{'source': 'anonymized', 'humanSpoiler': '2070...",[2070],"[[[0, 0], [0, 4]]]",phrase,[2070],Hole In Ozone Layer Expected To Make Full Reco...,"[[67, 71]]",0
2,008b7b19-0445-4e16-8f9e-075b73f80ca4,380537005123190784,This is what makes employees happy -- and it's...,Twitter,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[http://i.huffpost.com/gen/1359674/images/o-HA...,http://huff.to/1epfeaw,"{'source': 'anonymized', 'humanSpoiler': 'Inte...",[intellectual stimulation],"[[[1, 186], [1, 210]]]",phrase,[intellectual stimulation],Intellectual Stimulation Trumps Money For Empl...,"[[349, 373]]",0


In [14]:
from datasets import Dataset
train_df = generate_new_qa_form_df(train_df)
val_df = generate_new_qa_form_df(val_df)

train_data = Dataset.from_pandas(train_df.reset_index(drop=True), split="train")
val_data = Dataset.from_pandas(val_df.reset_index(drop=True), split="test")

In [15]:
train_df.head(2)

,id,title,question,context,answers
1,b1a1f63d-8853-4a11-89e8-6b2952a393ec,Hole In Ozone Layer Expected To Make Full Reco...,NASA sets date for full recovery of ozone hole,Hole In Ozone Layer Expected To Make Full Reco...,"{'text': ['2070'], 'answer_start': [67]}"
2,008b7b19-0445-4e16-8f9e-075b73f80ca4,Intellectual Stimulation Trumps Money For Empl...,This is what makes employees happy -- and it's...,Intellectual Stimulation Trumps Money For Empl...,"{'text': ['intellectual stimulation'], 'answer..."


In [16]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [17]:
train_data

Dataset({
    features: ['id', 'title', 'question', 'context', 'answers'],
    num_rows: 1367
})

In [18]:
train_dataset = train_data.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=train_data.column_names,
)
validation_dataset = val_data.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=val_data.column_names,
)


Map:   0%|          | 0/1367 [00:00<?, ? examples/s]

Map:   0%|          | 0/335 [00:00<?, ? examples/s]

In [19]:
import nltk
from collections import defaultdict
from tqdm.auto import tqdm
def compute_metrics(start_logits, end_logits, features, examples, predictOnly=False):

    example_to_features = defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_spoilers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        spoilers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    spoilers.append(answer)

        # Select the answer with the best score
        if len(spoilers) > 0:
            best_answer = max(spoilers, key=lambda x: x["logit_score"])
            predicted_spoilers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_spoilers.append({"id": example_id, "prediction_text": ""})
            
    predicted_texts = [i['prediction_text'] for i in predicted_spoilers]
    
    if predictOnly:
        return predicted_texts
    
    actual_spoilers_squad = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    actual_spoilers = [i['answers']['text'][0] for i in actual_spoilers_squad]    
    
    
    bleu_eval = bleu.compute(predictions=predicted_texts, references=actual_spoilers)
    meteor_eval = meteor.compute(predictions=predicted_texts,references = actual_spoilers)
    squad_metrics_eval = squad_metric.compute(predictions=predicted_spoilers, references=actual_spoilers_squad)
   
    return [squad_metrics_eval,meteor_eval,bleu_eval],actual_spoilers,predicted_texts

In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, RobertaTokenizer, AutoModelForQuestionAnswering,T5Tokenizer, T5ForConditionalGeneration


model_name = 'deepset/roberta-base-squad2'

#Phrase Generation
config = dict(
max_length = 512,
stride = 128,
n_best = 25,
max_answer_length = 30,
batch_size = 8,
epochs = 10,
learning_rate = 1e-6,
model_name = model_name,
spoiler_type = "phrase"
)



max_length = config["max_length"]
stride = config["stride"]
n_best = config["n_best"]
max_answer_length = config["max_answer_length"]


tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
model = AutoModelForQuestionAnswering.from_pretrained(config["model_name"])

# **ROBERTA FINE TUNING**

In [21]:
from transformers import TrainingArguments

args = TrainingArguments(
   
    model_name,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    
)

In [22]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)


In [23]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,1.446200,No log
2,0.733600,No log
3,0.357600,No log
4,0.202200,No log
5,0.122000,No log
6,0.071200,No log
7,0.049200,No log
8,0.028300,No log
9,0.017600,No log
10,0.015000,No log


TrainOutput(global_step=2970, training_loss=0.30430345776105167, metrics={'train_runtime': 2769.4031, 'train_samples_per_second': 8.579, 'train_steps_per_second': 1.072, 'total_flos': 6208410940047360.0, 'train_loss': 0.30430345776105167, 'epoch': 10.0})

In [24]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
metrics,theoretical_texts,predicted_texts = compute_metrics(
        start_logits, end_logits, validation_dataset, val_data
    )

  0%|          | 0/335 [00:00<?, ?it/s]

In [25]:
metrics

[{'exact_match': 59.1044776119403, 'f1': 70.78952829435708},
 {'meteor': 0.5803124230102755},
 {'bleu': 0.36639388010658286,
  'precisions': [0.5950323974082073,
   0.467005076142132,
   0.3159509202453988,
   0.20526315789473684],
  'brevity_penalty': 1.0,
  'length_ratio': 1.0843091334894615,
  'translation_length': 926,
  'reference_length': 854}]